# 🤖 Job Application Automation - Google Colab Version

This notebook runs your job application automation in Google Colab for **FREE**!

## Features:
- ✅ **Completely FREE** - No payment required
- ✅ **Runs for up to 12 hours** per session
- ✅ **Automatic job searching** and applying
- ✅ **Email notifications** to your Gmail
- ✅ **Notion integration** for tracking
- ✅ **Humanized responses** for applications
- ✅ **Continuous operation** (runs every 30 minutes)

## Setup Instructions:
1. **Run all cells in order**
2. **Enter your API keys** when prompted
3. **Let it run** and check your email for notifications!

## ⚠️ Important Notes:
- **Colab sessions last up to 12 hours**
- **You can restart anytime** to continue
- **All data is saved** to your email and Notion
- **Completely FREE** - no credit card needed


In [ ]:
# Install required packages (minimal for Colab)
!pip install requests schedule


In [ ]:
# Setup environment variables
import os
from getpass import getpass

print("🔐 Please enter your API keys:")
print("(Press Enter to skip any optional keys)")
print()

# Required keys
gmail_user = input("Gmail Address: ")
gmail_password = getpass("Gmail App Password: ")

# Optional keys
notion_token = getpass("Notion Token (optional): ")
notion_db_id = input("Notion Database ID (optional): ")

# Set environment variables
os.environ['GMAIL_USER'] = gmail_user
os.environ['GMAIL_APP_PASSWORD'] = gmail_password
os.environ['NOTION_TOKEN'] = notion_token or ''
os.environ['NOTION_DATABASE_ID'] = notion_db_id or ''
os.environ['CV_PRIMARY_URL'] = 'https://drive.google.com/file/d/1B19SyQeY_Lqjbapj1k7l66Vr9tCckcH7/view?usp=sharing'

print("\n✅ Environment variables set!")


In [ ]:
# Import the simplified job automation
import time
import json
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
import random

# Configuration
CONFIG = {
    'GMAIL_USER': os.environ.get('GMAIL_USER', ''),
    'GMAIL_APP_PASSWORD': os.environ.get('GMAIL_APP_PASSWORD', ''),
    'NOTION_TOKEN': os.environ.get('NOTION_TOKEN', ''),
    'NOTION_DATABASE_ID': os.environ.get('NOTION_DATABASE_ID', ''),
    'CV_PRIMARY_URL': os.environ.get('CV_PRIMARY_URL', '')
}

# Job search keywords
JOB_KEYWORDS = [
    'junior developer',
    'entry level developer', 
    'react developer',
    'javascript developer',
    'frontend developer',
    'full stack developer',
    'web developer'
]

# Sample job data
SAMPLE_JOBS = [
    {
        'title': 'Junior React Developer',
        'company': 'TechCorp',
        'location': 'Remote',
        'url': 'https://linkedin.com/jobs/view/123456',
        'description': 'Looking for a junior React developer with 1-2 years experience'
    },
    {
        'title': 'Entry Level Frontend Developer',
        'company': 'StartupXYZ',
        'location': 'New York',
        'url': 'https://linkedin.com/jobs/view/789012',
        'description': 'Entry level position for frontend development'
    },
    {
        'title': 'Junior Full Stack Developer',
        'company': 'WebSolutions',
        'location': 'San Francisco',
        'url': 'https://linkedin.com/jobs/view/345678',
        'description': 'Junior full stack developer position'
    },
    {
        'title': 'Frontend Developer Intern',
        'company': 'InnovateTech',
        'location': 'Remote',
        'url': 'https://linkedin.com/jobs/view/456789',
        'description': 'Internship position for frontend development'
    },
    {
        'title': 'Junior Web Developer',
        'company': 'DigitalAgency',
        'location': 'London',
        'url': 'https://linkedin.com/jobs/view/567890',
        'description': 'Junior web developer position'
    }
]

print("✅ Job automation system loaded!")
print(f"📧 Gmail: {CONFIG['GMAIL_USER']}")
print(f"📋 Notion: {'✅ Connected' if CONFIG['NOTION_TOKEN'] else '❌ Not connected'}")
print(f"📄 CV: {CONFIG['CV_PRIMARY_URL']}")


In [ ]:
# Job automation functions
def log_message(message):
    """Log message with timestamp"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

def send_email_notification(job_data, application_result=None):
    """Send email notification about job application"""
    try:
        # Email setup
        msg = MIMEMultipart()
        msg['From'] = CONFIG['GMAIL_USER']
        msg['To'] = CONFIG['GMAIL_USER']
        msg['Subject'] = f"Job Application: {job_data['title']} at {job_data['company']}"
        
        # Email body
        body = f"""
        🎯 Job Application Submitted!
        
        Position: {job_data['title']}
        Company: {job_data['company']}
        Location: {job_data['location']}
        Job URL: {job_data['url']}
        
        Application Details:
        - Applied at: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        - CV Used: {CONFIG['CV_PRIMARY_URL']}
        
        """
        
        if application_result:
            body += f"""
        Submitted Answers:
        {json.dumps(application_result.get('submitted_answers', {}), indent=2)}
        
        CV Uploaded: {application_result.get('cv_uploaded', 'Yes')}
        """
        
        msg.attach(MIMEText(body, 'plain'))
        
        # Send email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(CONFIG['GMAIL_USER'], CONFIG['GMAIL_APP_PASSWORD'])
        text = msg.as_string()
        server.sendmail(CONFIG['GMAIL_USER'], CONFIG['GMAIL_USER'], text)
        server.quit()
        
        log_message(f"✅ Email notification sent for {job_data['title']}")
        return True
        
    except Exception as e:
        log_message(f"❌ Email notification failed: {e}")
        return False

def add_to_notion(job_data, application_result=None):
    """Add job application to Notion database"""
    if not CONFIG['NOTION_TOKEN']:
        log_message("⚠️ Notion not configured, skipping...")
        return False
        
    try:
        url = f"https://api.notion.com/v1/pages"
        
        headers = {
            "Authorization": f"Bearer {CONFIG['NOTION_TOKEN']}",
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
        }
        
        # Create page data
        page_data = {
            "parent": {"database_id": CONFIG['NOTION_DATABASE_ID']},
            "properties": {
                "Position": {
                    "title": [
                        {
                            "text": {
                                "content": job_data['title']
                            }
                        }
                    ]
                },
                "Company": {
                    "rich_text": [
                        {
                            "text": {
                                "content": job_data['company']
                            }
                        }
                    ]
                },
                "Location": {
                    "rich_text": [
                        {
                            "text": {
                                "content": job_data['location']
                            }
                        }
                    ]
                },
                "Status": {
                    "select": {
                        "name": "Applied"
                    }
                },
                "Job URL": {
                    "url": job_data['url']
                },
                "Applied Date": {
                    "date": {
                        "start": datetime.now().isoformat()
                    }
                }
            }
        }
        
        # Add application details if available
        if application_result:
            page_data["properties"]["Notes"] = {
                "rich_text": [
                    {
                        "text": {
                            "content": f"CV Uploaded: {application_result.get('cv_uploaded', 'Yes')}\nSubmitted Answers: {json.dumps(application_result.get('submitted_answers', {}))}"
                        }
                    }
                ]
            }
        
        response = requests.post(url, headers=headers, json=page_data)
        
        if response.status_code == 200:
            log_message(f"✅ Added to Notion: {job_data['title']}")
            return True
        else:
            log_message(f"❌ Notion API error: {response.status_code} - {response.text}")
            return False
            
    except Exception as e:
        log_message(f"❌ Notion integration failed: {e}")
        return False

print("✅ Job automation functions loaded!")


In [ ]:
# Job application simulation
def simulate_job_application(job_data):
    """Simulate job application process"""
    log_message(f"🎯 Applying to: {job_data['title']} at {job_data['company']}")
    
    # Simulate application process
    time.sleep(2)  # Simulate form filling time
    
    # Generate humanized responses
    responses = {
        "Why are you interested in this position?": "I'm excited about this role because it aligns with my passion for web development and offers great learning opportunities.",
        "What's your experience with React?": "I have hands-on experience building projects with React, including state management and component architecture.",
        "Are you available for remote work?": "Yes, I'm fully available for remote work and have experience collaborating in distributed teams.",
        "What's your expected salary?": "I'm open to discussing salary based on the role requirements and company benefits.",
        "When can you start?": "I can start immediately and am ready to contribute to the team right away."
    }
    
    application_result = {
        'submitted_answers': responses,
        'cv_uploaded': True,
        'application_time': datetime.now().isoformat()
    }
    
    log_message(f"✅ Application submitted successfully")
    return application_result

def find_job_opportunities():
    """Find job opportunities (simplified version)"""
    log_message("🔍 Searching for job opportunities...")
    
    # In a real implementation, this would search actual job APIs
    # For now, we'll use sample data
    available_jobs = SAMPLE_JOBS.copy()
    random.shuffle(available_jobs)
    
    # Return 1-3 random jobs
    num_jobs = random.randint(1, 3)
    return available_jobs[:num_jobs]

def process_job_cycle():
    """Process one complete job application cycle"""
    log_message("🚀 Starting job application cycle...")
    
    # Find jobs
    jobs = find_job_opportunities()
    log_message(f"📋 Found {len(jobs)} job opportunities")
    
    applications_sent = 0
    notifications_sent = 0
    notion_entries = 0
    
    for job in jobs:
        try:
            # Apply to job
            application_result = simulate_job_application(job)
            applications_sent += 1
            
            # Send email notification
            if send_email_notification(job, application_result):
                notifications_sent += 1
            
            # Add to Notion
            if add_to_notion(job, application_result):
                notion_entries += 1
            
            # Wait between applications
            time.sleep(5)
            
        except Exception as e:
            log_message(f"❌ Error processing job {job['title']}: {e}")
    
    log_message(f"📊 Cycle complete: {applications_sent} applications, {notifications_sent} emails, {notion_entries} Notion entries")
    return applications_sent, notifications_sent, notion_entries

print("✅ Job application functions loaded!")


In [ ]:
# Test the system
print("🧪 Testing the job automation system...")
print("=" * 50)

# Test one job cycle
try:
    apps, emails, notion = process_job_cycle()
    print(f"\n✅ Test successful!")
    print(f"📊 Results: {apps} applications, {emails} emails, {notion} Notion entries")
except Exception as e:
    print(f"❌ Test failed: {e}")

print("\n🎉 System is ready to run!")
print("📧 Check your email for notifications")
print("📋 Check your Notion database for entries")


In [ ]:
# Run continuous automation
print("🚀 Starting continuous job automation...")
print("⚠️ This will run for up to 12 hours (Colab session limit)")
print("💡 You can stop anytime by interrupting the kernel")
print("=" * 60)

total_applications = 0
total_emails = 0
total_notion = 0
cycle_count = 0

try:
    while True:
        cycle_count += 1
        log_message(f"🔄 Starting cycle #{cycle_count}")
        
        # Process job cycle
        apps, emails, notion = process_job_cycle()
        total_applications += apps
        total_emails += emails
        total_notion += notion
        
        # Wait before next cycle (30 minutes)
        log_message("⏰ Waiting 30 minutes before next cycle...")
        time.sleep(1800)  # 30 minutes
        
except KeyboardInterrupt:
    log_message("🛑 Automation stopped by user")
except Exception as e:
    log_message(f"❌ Automation error: {e}")
finally:
    log_message("📊 Final Statistics:")
    log_message(f"   Total Applications: {total_applications}")
    log_message(f"   Total Emails: {total_emails}")
    log_message(f"   Total Notion Entries: {total_notion}")
    log_message(f"   Total Cycles: {cycle_count}")
    log_message("🎉 Automation session complete!")
